In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [3]:
#------------------80-chars----------------------------------------------------|
def train_car_classifier(img_paths):
    """
    Trains a classifier to recognize cars
    
    :param img_paths: A list of image filepaths
    :return classifier: A classifier that detects cars 
    """
    return classifier

In [4]:
#------------------80-chars----------------------------------------------------|
def crop_image(img, x_usable=(0.0,1.0), y_usable=(0.4,1.0)):
    """
    Crop parts of the image where cars should not exist
    
    :param img: Image to be cropped
    :param x_usable: Tuple - Fraction of image 0-1 in x start-end that's usable
    :param y_usable: Tuple - Fraction of image 0-1 in y start-end that's usable
    """
    x_start = np.int(img.shape[0]*x_usable[0])
    x_end   = np.int(img.shape[0]*x_usable[1])
    y_start = np.int(img.shape[1]*y_usable[0])
    y_end   = np.int(img.shape[1]*y_usable[1])
    return img[x_start:x_end,y_start:y_end,:]

In [ ]:
#------------------80-chars----------------------------------------------------|
def get_hog_features(img, pix_per_cell, cell_per_block, orient):
    """
    Extract histogram of gradient features from an image
    
    :param img: Image to extract features from
    :return features: HOG feature vector
    """
    ch_0 = img[:,:,0]
    ch_1 = img[:,:,1]
    ch_2 = img[:,:,2]
    
    hog_0 = get_hog_features(ch_0, orient, pix_per_cell, cell_per_block,
                             feature_vec=False)
    hog_1 = get_hog_features(ch_1, orient, pix_per_cell, cell_per_block,
                             feature_vec=False)
    hog_2 = get_hog_features(ch_2, orient, pix_per_cell, cell_per_block,
                             feature_vec=False)
    #TODO: Iterate over hog_* and extract hog_feature vectors
    return features

In [2]:
def pipeline(img):
    """
    Pipeline detects vehicles in an image
    
    :param img: Image to detect cars in
    :return final_img: Image with bounding boxes around cars
    """
    return final_img

In [ ]:
train_car_classifier()